# Migrating to Scipp 23.01.0 

## Overview

Starting with Scipp 23.01.0, a `VariancesError` is raised when an operand with variances (statistical uncertianties) is implicitly or explicitly broadcast.
This is a necessary bugfix, since Scipp is not capable of tracking correlations and would thus silently underestimate uncertainties.

## What is affected?

Any operation with a broadcast of an operand with variances is affected.
This can happen for dense data as well as for binned data.
We give two examples:

In [ ]:
import scipp as sc

var = sc.ones(dims=['x', 'y'], shape=(4, 3))
norm = sc.array(
    dims=['y'],
    values=[0.1, 0.2, 0.3],
    variances=[0.1, 0.2, 0.3],
)

An operation between `var` and `norm` will raise `VariancesError` since `norm` is broadcast into dimension `'x'`:

In [ ]:
var / norm

In [ ]:
binned = sc.data.table_xyz(100).bin(y=3)

An operation between `binned` and `norm` will raise `VariancesError` since `norm` is broadcast to all bin elements:

In [ ]:
binned / norm

## Working around the new behavior

Scipp will not be able to support an automatic approach for handling these cases correctly, since this would be either very costly or very problem-specific.
An easy **but wrong** work around (equivalent to the old behavior in almost all cases) is to set the variances of the term being broadcast to `None`:

In [ ]:
import logging

stream_handler = logging.StreamHandler()
logger = sc.get_logger()
logger.addHandler(stream_handler)

In [ ]:
logger.warning(
    """ATTENTION:
    Statistical uncertainties of the normalization term were IGNORED.
    This is probably incorrect."""
)
norm.variances = None
var / norm
binned / norm

We repeat that this is **incorrect**.
We strongly recommend to issue an explicit warning to the user, as shown in the example.

## Fixing the problem

For many applications there is no simple solution to this, as it may involve an intractably large correlation matrix.
See our publication (preprint submitted to Journal of Neutron Science) for solutions in certain cases.

We recommend to derive the actual expression for uncertainties of the entire calculation, using a first-order truncated Taylor expansion.
In a number of our own applications it turned out that the final expression can be computed rather cheaply &mdash; without dealing with the intractably large intermediate correlation matrices.

If this is not possible, approaches such as Bootstrapping may be used.